In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# Building an IMDB Top 250 clone with pandas
Building a simple recommender system using weighted average

In [3]:
df=pd.read_csv('movies_metadata.csv')

C:\Users\Vishnu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


# Metric 

Metric used is arbitary.Movie is considered to be good if it has a rating higher than other movies.We have to be very careful while choosing the right metric since it affects our recommendation system.

The simplest metric we can use here is the rating .

Disadvantages:
It does not take in account of the popularity. A movie rated 8 by 100,000 users will be placed below a movie rated 8.5 by 100 users.We can say that movie rated by 100 users caters to a specific audience .


As the number of voters increase,the rating of a movie normalizes and it approaches a value which is reflects the movies quality.Means,movies with a few rating doesn't mean it is a good movie ex.(10/10 by a 10 users)


# Weighted average
Let us build a simple recommender system using IMDB's weighted average formula.


<b>Weighted Rating(WR)=(v/(v+m) * R) + (m/(v+m)) * C)<b>
    
    v= is the no of votes of the movie
    m= minimun no of votes required to be in the list 
    R= mean rating of the movie 
    C= mean rating of all movies
 

In [28]:
#only 40 % percent of people have gathered more than 15 
m=df['vote_count'].quantile(0.75)
print(m)

34.0


The list includes documentaries,short films etc.inorder the recommend movies we will filter the movies according to the runtime

In [29]:
q_movies=df[(df['runtime']>45) & (df['runtime']<200)]
q_movies.shape

(41622, 24)

In [30]:
# taking out movies who has more the 25 user votes 
q_movies=q_movies[q_movies['vote_count']>m]
q_movies.shape

(10977, 24)

In [31]:
C=df['vote_average'].mean()

In [32]:
#we can se that out data size has considerably reduced with all that filtering 
#let's calculate the noramilized score for each movie 
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Compute the weighted score
    return (v/(v+m) * R) + (m/(m+v) * C)


In [33]:
#lets compute the score for each movies
q_movies['score']=q_movies.apply(weighted_rating,axis=1)

In [34]:
q_movies[['title','vote_count','vote_average','score','runtime']].sort_values('score',ascending=False).head(10)

,title,vote_count,vote_average,score,runtime
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.929668,190.0
314,The Shawshank Redemption,8358.0,8.5,8.488324,142.0
834,The Godfather,6024.0,8.5,8.483826,175.0
40251,Your Name.,1030.0,8.5,8.407913,106.0
12481,The Dark Knight,12269.0,8.3,8.292589,152.0
2843,Fight Club,9678.0,8.3,8.290612,139.0
292,Pulp Fiction,8670.0,8.3,8.289524,154.0
522,Schindler's List,4436.0,8.3,8.279602,195.0
23673,Whiplash,4376.0,8.3,8.279324,105.0
5481,Spirited Away,3968.0,8.3,8.277216,125.0


As the values of m changes the chart changes too 
When we keep low values of m we can see that dilwale dulhaniya is on top but the vote vote count is less compared to other movies,if we try to change the values of m we can see that the chart changes and higher vote count movies comes up.


the baseline we set for minimum movie votes is critical.
